In [39]:
!pip install ipython==7.34.0 ipykernel==5.5.6
!pip install import_ipynb

In [40]:
try:
    import google.colab  # noqa: F401
except ImportError:
    import dolfin
else:
    try:
        import dolfin
    except ImportError:
        !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
        import dolfin

import numpy as np
import matplotlib.tri as tri
import fenics as fe

In [41]:
from google.colab import drive
drive.mount('/content/drive')
import os
import import_ipynb

path = '/content/drive/MyDrive/Colab Notebooks/Physics-Informed Neural Networks/Demo/fenics_cfd/generate_data'
os.chdir(path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [42]:
from generate_flow_fields import define_function_spaces

In [43]:
def get_magnitude(u_values):
  return np.sqrt(np.sum(u_values**2, axis=1))

In [44]:
def get_triangulation(coordinates, connectivity):
  return tri.Triangulation(coordinates[:, 0], coordinates[:, 1], connectivity)

In [45]:
def get_domain_dimensions(domain_data):

  if isinstance(domain_data, fe.Mesh):
      x_coords = domain_data.coordinates()[:, 0]
      y_coords = domain_data.coordinates()[:, 1]
  elif isinstance(domain_data, np.ndarray):
      x_coords = domain_data[:, 0]
      y_coords = domain_data[:, 1]

  length = x_coords.max() - x_coords.min()
  height = y_coords.max() - y_coords.min()

  return length, height

In [46]:
def convert_to_numpy(mesh, u, p):

  coordinates = mesh.coordinates()
  connectivity = np.asarray([cell.entities(0) for cell in fe.cells(mesh)])
  triangulation = get_triangulation(coordinates, connectivity)

  u_values = u.compute_vertex_values(mesh).reshape((2, coordinates.shape[0])).T
  p_values = p.compute_vertex_values(mesh)

  return u_values, p_values, coordinates, connectivity

In [47]:
def generate_coarse_mesh(length, height, resolution):
  return fe.RectangleMesh(fe.Point(0.0, 0.0), fe.Point(length, height), resolution, int(np.round(height / length * resolution)))

In [48]:
def interpolate_to_coarse_mesh(mesh, u, p, coarse_mesh, coarse_resolution):

  V_coarse, Q_coarse = define_function_spaces(coarse_mesh)

  u.set_allow_extrapolation(True)
  p.set_allow_extrapolation(True)

  u_coarse = fe.interpolate(u, V_coarse)
  p_coarse = fe.interpolate(p, Q_coarse)

  return u_coarse, p_coarse

In [49]:
def generate_coarse_fields(mesh, u, p, coarse_resolution):

  max_length, max_height = get_domain_dimensions(mesh)

  coarse_mesh = generate_coarse_mesh(max_length, max_height, coarse_resolution)
  u_coarse, p_coarse = interpolate_to_coarse_mesh(mesh, u, p, coarse_mesh, coarse_resolution)
  u_coarse_values, p_coarse_values, coarse_coordinates, coarse_connectivity = convert_to_numpy(coarse_mesh, u_coarse, p_coarse)
  u_coarse_values_masked, p_coarse_values_masked = mask_field_values(mesh, u_coarse_values, p_coarse_values, coarse_mesh)

  return u_coarse_values_masked, p_coarse_values_masked, coarse_coordinates, coarse_connectivity

In [50]:
def is_inside_vessel(point, mesh):

    pt = fe.Point(*point)
    inside = mesh.bounding_box_tree().compute_first_entity_collision(pt) < mesh.num_cells()

    return inside

In [51]:
def mask_field_values(mesh, u_values, p_values, coarse_mesh):
  coarse_coordinates = coarse_mesh.coordinates()

  mask = np.array([is_inside_vessel(pt, mesh) for pt in coarse_coordinates])

  u_masked = np.where(np.repeat(mask[:, np.newaxis], 2, axis=1), u_values, 0)
  p_masked = np.where(mask, p_values, 0)

  return u_masked, p_masked

In [52]:
def add_noise_to_fields(u_values, p_values, sigma_u, sigma_p):

  u_noisy = u_values + np.random.normal(0, sigma_u, u_values.shape)
  p_noisy = p_values + np.random.normal(0, sigma_p, p_values.shape)

  return u_noisy, p_noisy

In [57]:
def convert_to_matrix(u_values, p_values, resolution, flip_bool):

    u_x = u_values[:, 0]  # Extract x-component
    u_y = u_values[:, 1]  # Extract y-component

    u_x_reshaped = u_x.reshape(-1, resolution)
    u_y_reshaped = u_y.reshape(-1, resolution)
    p_reshaped = p_values.reshape(-1, resolution)

    if flip_bool:
      u_x_reshaped = np.flip(u_x_reshaped, axis=0)
      u_y_reshaped = np.flip(u_y_reshaped, axis=0)
      p_reshaped = np.flip(p_reshaped, axis=0)

    u_reshaped = np.stack((u_x_reshaped, u_y_reshaped), axis=-1)

    return u_reshaped, p_reshaped

In [76]:
def zero_pad_to_square(u_values, p_values, target_size, flip_bool):
    u_reshaped, p_reshaped = convert_to_matrix(u_values, p_values, target_size, flip_bool)

    N, M = u_reshaped.shape[:2]

    if N > M:
        excess_height = N - M
        trim_top = excess_height // 2
        trim_bottom = excess_height - trim_top
        u_reshaped = u_reshaped[trim_top:N - trim_bottom]
        p_reshaped = p_reshaped[trim_top:N - trim_bottom]

    N, M = u_reshaped.shape[:2]

    top_pad = (target_size - N) // 2
    bottom_pad = target_size - N - top_pad

    padded_u_values = np.pad(u_reshaped, ((top_pad, bottom_pad), (0, 0), (0, 0)), mode='constant', constant_values=0)
    padded_p_values = np.pad(p_reshaped, ((top_pad, bottom_pad), (0, 0)), mode='constant', constant_values=0)

    return padded_u_values, padded_p_values

In [77]:
def generate_final_vessel_data(mesh, u, p, resolution, noise_level, flip_bool):

  u_values, p_values, _, _ = generate_coarse_fields(mesh, u, p, resolution-1)
  u_square_matrix, p_square_matrix = zero_pad_to_square(u_values, p_values, resolution, flip_bool)

  u_noise_scale = np.max(get_magnitude(u_values))
  p_noise_scale = np.max(p_values)

  u_noisy_np, p_noisy_np = add_noise_to_fields(u_square_matrix, p_square_matrix, sigma_u=u_noise_scale*noise_level, sigma_p=p_noise_scale*noise_level)

  return u_noisy_np, p_noisy_np

In [71]:
def generate_hr_and_lr_data(mesh, u, p, high_resolution, low_resolution, noise_level):

  flip_bool = np.random.rand() < 0.5    # Determine whether to flip or not
  u_hr, p_hr = generate_final_vessel_data(mesh, u, p, high_resolution, 0, flip_bool)
  u_lr, p_lr = generate_final_vessel_data(mesh, u, p, low_resolution, noise_level, flip_bool)

  return u_hr, p_hr, u_lr, p_lr